<a href="https://colab.research.google.com/github/agemagician/Prot-Transformers/blob/master/Embedding/Advanced/Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3> Extracting protein sequences' features using ProtBert pretrained-model <h3>

<b>1. Load necessry libraries including huggingface transformers<b>

In [1]:
!pip install -q transformers
!pip install -q gdown

     |████████████████████████████████| 675kB 4.6MB/s 
     |████████████████████████████████| 1.1MB 21.9MB/s 
     |████████████████████████████████| 3.8MB 16.1MB/s 
     |████████████████████████████████| 890kB 54.9MB/s 


In [17]:
import torch
from transformers import BertModel, BertTokenizer
import re
import os
import gdown
from time import sleep

<b>2. Set the url location of ProtBert and the vocabulary file<b>

In [20]:
modelUrl = 'https://drive.google.com/uc?export=download&id=1mLuVhMuGTSSfVkK1rrmBSTC5yiCpGy--'
configUrl = 'https://drive.google.com/uc?export=download&id=1hg30JtXz6Okl0esJnMC2J_9TNbx5gJpl'
vocabUrl = 'https://drive.google.com/uc?export=download&id=15eFspbhoF5uUZ6xKKTAIGOunXYDeFruL'

<b>3. Download ProtBert models and vocabulary files<b>

In [21]:
downloadFolderPath = 'models/ProtBert/'

In [22]:
modelFolderPath = downloadFolderPath

modelFilePath = os.path.join(modelFolderPath, 'pytorch_model.bin')

configFilePath = os.path.join(modelFolderPath, 'config.json')

vocabFilePath = os.path.join(modelFolderPath, 'vocab.txt')

In [23]:
if not os.path.exists(modelFolderPath):
    os.makedirs(modelFolderPath)

In [24]:
def download_file(url,filename):
  while not os.path.exists(filename):
    sleep(1) # Time in seconds
    gdown.download(url,filename, quiet=False)

In [25]:
if not os.path.exists(modelFilePath):
    download_file(modelUrl, modelFilePath)

if not os.path.exists(configFilePath):
    download_file(configUrl, configFilePath)

if not os.path.exists(vocabFilePath):
    download_file(vocabUrl, vocabFilePath)

Permission denied: https://drive.google.com/uc?export=download&id=1mLuVhMuGTSSfVkK1rrmBSTC5yiCpGy--
Maybe you need to change permission over 'Anyone with the link'?
Permission denied: https://drive.google.com/uc?export=download&id=1mLuVhMuGTSSfVkK1rrmBSTC5yiCpGy--
Maybe you need to change permission over 'Anyone with the link'?


KeyboardInterrupt: ignored

<b>4. Load the vocabulary and ProtBert Model</b>

In [ ]:
tokenizer = BertTokenizer(vocabFilePath, do_lower_case=False )

In [ ]:
model = BertModel.from_pretrained(modelFolderPath)

<b>5. Load the model into the GPU if avilabile and switch to inference mode<b>

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = model.to(device)
model = model.eval()

<b>6. Create or load sequences and map rarely occured amino acids (U,Z,O,B) to (X)<b>

In [ ]:
sequences_Example = ["A E T C Z A O","S K T Z P"]

In [ ]:
sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]

<b>7. Tokenize, encode sequences and load it into the GPU if possibile<b>

In [ ]:
ids = tokenizer.batch_encode_plus(sequences_Example, add_special_tokens=True, pad_to_max_length=True)

In [ ]:
input_ids = torch.tensor(ids['input_ids']).to(device)
attention_mask = torch.tensor(ids['attention_mask']).to(device)

<b>8. Extracting sequences' features and load it into the CPU if needed<b>

In [ ]:
with torch.no_grad():
    embedding = model(input_ids=input_ids,attention_mask=attention_mask)[0]

In [ ]:
embedding = embedding.cpu().numpy()

<b>9. Remove padding ([PAD]) and special tokens ([CLS],[SEP]) that is added by Bert model<b>

In [ ]:
features = [] 
for seq_num in range(len(embedding)):
    seq_len = (attention_mask[seq_num] == 1).sum()
    seq_emd = embedding[seq_num][1:seq_len-1]
    features.append(seq_emd)

In [ ]:
print(features)